# PageRank

In [ ]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()

In [ ]:
gds.run_cypher("""
               CALL gds.graph.project(
                   'popularQuestionAlternatives',
                   ['QuestionAlternative', 'Respondent'],
                   {
                        CHOSE: {
                            orientation: 'REVERSE'
                        }
                   }
               )
               """)

In [ ]:
gds.run_cypher("""
               CALL gds.degree.write.estimate('popularQuestionAlternatives', 
               { writeProperty: 'degree' })
               YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
               """)

In [ ]:
gds.run_cypher("""
               CALL gds.graph.relationshipProperties.write('popularQuestionAlternatives', {
                     relationshipProperties: {
                            CHOSE: {
                                property: 'weight',
                                defaultValue: 1.0
                            }
               """)

In [ ]:
gds.run_cypher("""
               CALL gds.degree.mutate('popularQuestionAlternatives',
                { mutate })
                YIELD nodeId, score
                WHERE score > 0
                RETURN gds.util.asNode(nodeId).name AS name, score AS respondents
                ORDER BY respondents ASC, name ASC
               """)

In [ ]:
gds.run_cypher("""
               CALL gds.degree.stats('popularQuestionAlternatives')
               YIELD centralityDistribution
               RETURN centralityDistribution.min as minimumScore, centralityDistribution.max as maximumScore, centralityDistribution.mean as meanScore
               """)


In [ ]:
gds.run_cypher("""
               CALL gds.degree.stream(
                    'popularQuestionAlternatives',
                    { relationshipWeightProperty: 'score' }
               )
               YIELD nodeId, score
               RETURN gds.util.asNode(nodeId).name AS name, score AS weightedRespondents
               ORDER by weightedRespondents DESC, name ASC
               """)